# Task in this notebook

**List of tasks**
* EDA (Some EDA)
* Image PreProcessing (like augmentation)
* Divided dataset into
* * Train Dataset
* * Validation Dataset
* Basic Modeling Without Pre-Trained and check Loss curve
* Add pre_trained Model and check loss curve
* Some Tuning (if possible)

# 1. Setting basic configurations

In [1]:
from IPython.display import clear_output
!pip install -qU scikit-learn
clear_output()

In [2]:
%config Completer.use_jedi = False
import os, re, math, random, shutil, warnings, gc
import glob
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import cv2

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
from matplotlib.ticker import StrMethodFormatter
from mpl_toolkits.axes_grid1 import ImageGrid
from kaggle_datasets import KaggleDatasets
from kaggle_secrets import UserSecretsClient

from sklearn.model_selection import train_test_split

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.options.mode.chained_assignment = None
tf.get_logger().setLevel('ERROR')
py.init_notebook_mode(connected=True)


/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.



In [3]:
#if os.path.exists('/kaggle/working/Training_Images/Non_cancerous'):
    #os.remove('/kaggle/working/Training_Images/Non_cancerous')
   # print("File removed successfully")

In [4]:
devices = ['TPU', 'GPU']
img_size = [(1024,512), (512,256)]
img_ext = ['png']

class Config:
    
    def __init__(self):
        
        self.project_name="RSNA_BCD_Modeling"
        self.device=devices[0]
        self.seed=150
        
        self.path='/kaggle/input/rsna-bcs-imagepreprosessing/'
        self.training_imgs= self.path+'train_images'
        
        self.training_images_path='/kaggle/working/Training_Images/'
        self.validations_images_path='/kaggle/working/Validation_Images/'
        self.cancerous='Cancerous/'
        self.non_cancerous='Non_cancerous/'
        self.train_csv=self.path+'TrainDataset.csv'
        self.img_size = img_size[1]
        self.img_ext = img_ext[0]
        

config=Config()
        

# 2. Seeding Random Generators

In [5]:
def seeding(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    
seeding(config.seed)

# 3.Configur devices

In [6]:
# if config.device == 'TPU':

#     print('Connecting to TPU...')
#     try:
#         tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() 
#     except ValueError:
#         raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

#     tf.keras.mixed_precision.set_global_policy("mixed_bfloat16")
#     tf.config.set_soft_device_placement(True)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)

#     #config.path = KaggleDatasets().get_gcs_path(config.path.split('/')[3])
#     #config.train_path = config.path + '/' + 'train_images/'
#     #config.num_devices = strategy.num_replicas_in_sync
#     #config.parameters['batch_size'] = config.parameters['batch_size'] * config.num_devices

#     #print(f'Running on {config.num_devices} TPU devices')


# if config.device == 'GPU':
    
#     num_devices = len(tf.config.list_physical_devices('GPU'))

#     if num_devices > 1:
#         config.num_devices = num_devices
#         strategy = tf.distribute.MirroredStrategy()
#         #config.parameters['batch_size'] = config.parameters['batch_size'] * config.num_devices
#         #print(f'Running on {num_devices} GPU devices')
#     elif num_devices == 1:
#         strategy = tf.distribute.get_strategy()
#         print(f'Running on {num_devices} GPU device')
#     else:
#         strategy = tf.distribute.get_strategy()
#         config.device = 'CPU'
#         print(f'Running on CPU')
    
#     tf.config.optimizer.set_jit(True)
#     tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [7]:
#shutil.rmtree(config.training_images_path+config.non_cancerous)
#shutil.rmtree(config.training_images_path+config.cancerous)

#shutil.rmtree(config.validations_images_path+config.non_cancerous)
#shutil.rmtree(config.validations_images_path+config.cancerous)

In [8]:
# Create Training set directories for both classes
Training_cancer_dir = os.path.join(config.training_images_path, config.cancerous)
os.makedirs(os.path.dirname(Training_cancer_dir),exist_ok=True)

# Create the "No Cancer" directory
Training_no_cancer_dir = os.path.join(config.training_images_path,config.non_cancerous)
os.makedirs(os.path.dirname(Training_no_cancer_dir),exist_ok=True)


In [9]:
# Create Validations set directories for both classes
Validation_cancer_dir = os.path.join(config.validations_images_path, config.cancerous)
os.makedirs(os.path.dirname(Validation_cancer_dir),exist_ok=True)

# Create the "No Cancer" directory
Validations_no_cancer_dir = os.path.join(config.validations_images_path,config.non_cancerous)
os.makedirs(os.path.dirname(Validations_no_cancer_dir),exist_ok=True)

In [10]:
train_df = pd.read_csv(config.train_csv)
train_df.head()

,Unnamed: 0,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,input_image_path,output_image_path
0,0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10006/462822612.png
1,1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10006/1459541791.png
2,2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10006/1864590858.png
3,3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10006/1874946579.png
4,4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10011/220375232.png


In [11]:
test_image_dirs = glob.glob(config.path+"test_images/*/*.png")
train_image_dirs = glob.glob(config.path+"train_images/*/*.png")

images_Cancer=[]
images_noCancer=[]

print(train_df.shape)
i=0
for image in train_image_dirs:
    
    imageId=str(image.split('/')[6].split('.')[0])
    
  
    cancer_status=str(train_df.loc[train_df["image_id"].astype(str)==imageId].iloc[0]["cancer"])
    
    if(cancer_status=='1'):
        images_Cancer.append(imageId)
    else:
        images_noCancer.append(imageId)

print("Number of training images with Cancer:",len(images_Cancer))
print("number of training images with no cancer:",len(images_noCancer))
print("total number of training images : ",len(train_image_dirs))
print("Data type of image no cancer :",type(images_noCancer))

(54706, 17)
Number of training images with Cancer: 1158
number of training images with no cancer: 7465
total number of training images :  8623
Data type of image no cancer : <class 'list'>


In [12]:

images_noCancer_=images_noCancer[:2000]
print(len(images_noCancer_))

2000


# 4. Take 2000 images of non cancereous and all the cancereous images for model training

In [13]:
datagen = ImageDataGenerator()

# Get the filenames and labels of the images in the source directory
data = datagen.flow_from_directory(
        directory=config.path,
        batch_size=32,
        class_mode='binary',
        shuffle=False)


Found 8627 images belonging to 2 classes.


In [14]:
cancereous_images =[]
non_cancereous_images=[]
for i in range(len(data.filenames)):
    
    if str(data.filenames[i].split('/')[0])=='train_images':
        
        if str(data.filenames[i].split('/')[2].split('.')[0]) in images_noCancer_:
            img = cv2.imread(str(config.path)+data.filenames[i])
            img = cv2.resize(img, img_size[1][::-1], interpolation = cv2.INTER_AREA)
            non_cancereous_images.append(np.array(img))
           # plt.imshow(cancereous_images[0])
            
                
            #shutil.copy(os.path.join(config.path,data.filenames[i]), no_cancer_dir)
        elif str(data.filenames[i].split('/')[2].split('.')[0]) in images_Cancer:
            img = cv2.imread(str(config.path)+data.filenames[i])
            img = cv2.resize(img, img_size[1][::-1], interpolation = cv2.INTER_AREA)
            cancereous_images.append(np.array(img))
            #plt.imshow(cancereous_images[0])
            #shutil.copy(os.path.join(config.path,data.filenames[i]),cancer_dir)
        else:
            continue
            

In [15]:
print(len(cancereous_images))
print(len(non_cancereous_images))

1158
2000


**Find the class to be augmented**

In [16]:
# Select the images belonging to the class to augment
non_cancereous_images=np.array(non_cancereous_images)
cancereous_images = np.array(cancereous_images)
n_augment = len(non_cancereous_images)-len(cancereous_images)
idx_to_aug = np.random.randint(cancereous_images.shape[0], size=n_augment)
print("number of augumented images :",n_augment)




number of augumented images : 842


In [17]:
# split the data into training and validation sets
train_non_cancereous, val_non_cancereous = train_test_split(non_cancereous_images, 
                                                            test_size=0.2, 
                                                            random_state=config.seed)

# create directories for the training and validation sets
for i, img in enumerate(train_non_cancereous):
    save_path = os.path.join(Training_no_cancer_dir, 'non_cancer_' + str(i) + '.png')
    cv2.imwrite(save_path, img)
    
for i, img in enumerate(val_non_cancereous):
    save_path = os.path.join(Validations_no_cancer_dir, 'non_cancer_' + str(i) + '.png')
    cv2.imwrite(save_path, img)



In [18]:
non_cancereous_images=[]

In [19]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ])

In [20]:
augumented_images = []

plt.figure(figsize=(10, 10))
for i in range(n_augment):
    augmented_image = data_augmentation(cancereous_images[idx_to_aug[i]])
    augumented_images.append(augmented_image)
    
    
#     if i<4:
#         ax = plt.subplot(2, 2, i + 1)
#         plt.imshow(augmented_image)
#         plt.axis("off")
#         #plt.imshow(cancereous_images[0])
    
        

<Figure size 1000x1000 with 0 Axes>

In [21]:

cancereous_images=np.append(cancereous_images, augumented_images,axis=0)

print("Number of cancereous images : ",len(cancereous_images))

Number of cancereous images :  2000


In [22]:

train_cancereous, val_cancereous = train_test_split(cancereous_images, 
                                                    test_size=0.2, 
                                                    random_state=config.seed)

# save the cancereous images into training and validation sets of their respective directories
for i, img in enumerate(train_cancereous):
    save_path = os.path.join(Training_cancer_dir, 'cancer_' + str(i) + '.png')
    cv2.imwrite(save_path, img)



for i, img in enumerate(val_cancereous):
    save_path = os.path.join(Validation_cancer_dir, 'cancer_' + str(i) + '.png')
    cv2.imwrite(save_path, img)

cancereous_images=[]
print("Number of cancereous images : ",len(cancereous_images))
    

Number of cancereous images :  0


In [23]:
# # set up data generators for training and validation sets
# train_datagen = ImageDataGenerator(rescale=1./255)
# train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         class_mode='binary')

# validation_generator = train_datagen.flow_from_directory(
#         val_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         class_mode='binary')

In [24]:

# # concatenate the two arrays
# all_images = np.concatenate((all_images,cancereous_images), axis=0)

# # create labels for the images (0 for non-cancerous, 1 for cancerous)
# all_labels = np.concatenate((np.zeros(len(non_cancereous_images)), np.ones(len(cancereous_images))))

# # split the data into training and validation sets
# train_images, val_images, train_labels, val_labels = train_test_split(all_images, 
#                                                                       all_labels, 
#                                                                       test_size=0.2, 
#                                                                       random_state=config.seed)
# # normalize the pixel values of the images to be between 0 and 1
# #train_images = train_images / 255.0
# #val_images = val_images / 255.0


In [25]:
# print(train_images[0])

In [26]:
# BCD_model = Sequential()

# BCD_model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(img_size[1][::-1],3)))
# BCD_model.add(MaxPool2D(pool_size=3, strides=2))

# BCD_model.add(Conv2D(64, kernel_size=3, activation='relu'))
# BCD_model.add(MaxPool2D(pool_size=3, strides=2))

# BCD_model.add(Conv2D(128, kernel_size=3, activation='relu'))
# BCD_model.add(MaxPool2D(pool_size=3, strides=2))

# BCD_model.add(Flatten())
# BCD_model.add(Dropout(0.2))
# BCD_model.add(Dense(128, activation='relu'))
# BCD_model.add(Dense(1, activation='sigmoid', name='gender'))



In [27]:
# gender_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [28]:
# history_gender = gender_model.fit(x_train_gender, y_train_gender,
#                         validation_data=(x_test_gender, y_test_gender), epochs=10)

In [29]:

# ###########################
# gender_model.save(save_model_path+'gender_model.h5')
# ###########################

In [30]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               54706 non-null  int64  
 1   site_id                  54706 non-null  int64  
 2   patient_id               54706 non-null  int64  
 3   image_id                 54706 non-null  int64  
 4   laterality               54706 non-null  object 
 5   view                     54706 non-null  object 
 6   age                      54669 non-null  float64
 7   cancer                   54706 non-null  int64  
 8   biopsy                   54706 non-null  int64  
 9   invasive                 54706 non-null  int64  
 10  BIRADS                   26286 non-null  float64
 11  implant                  54706 non-null  int64  
 12  density                  29470 non-null  object 
 13  machine_id               54706 non-null  int64  
 14  difficult_negative_cas

# 3. Exploratory Data Analysis (EDA)

In [31]:
fig = go.Figure(data=[go.Pie(labels=['Non cancerous', 'Cancerous'], values=train_df['cancer'].value_counts(), hole=.3)])
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(title_text='Number of patients with cancer', title_x=0.5)
fig.show()


In [32]:
fig = px.histogram(train_df, x="age", title='Age distribution of the patients')
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_x=0.5)
fig.show()

In [33]:

fig = px.violin(train_df.sample(frac=0.1), y="age", x="cancer", color="cancer", box=True, 
                points="all",
                title='Age distribution with respect to the cancer')
fig.update_layout(title_x=0.5)
fig.show()


In [34]:
fig = px.histogram(train_df.groupby('patient_id').count().reset_index(), x="image_id", color='image_id',
                   title='Number of images per patient')
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_x=0.5)
fig.show()

In [35]:
fig = make_subplots(rows=2, cols=2, subplot_titles=('Laterality', 'Density', 'View', 'Implant'))

fig.add_trace(go.Bar(x=train_df['laterality'].value_counts().index, y=train_df['laterality'].value_counts().values), row=1, col=1)
fig.add_trace(go.Bar(x=train_df['density'].value_counts().index, y=train_df['density'].value_counts().values), row=1, col=2)
fig.add_trace(go.Bar(x=train_df['view'].value_counts().index, y=train_df['view'].value_counts().values), row=2, col=1)
fig.add_trace(go.Bar(x=train_df['implant'].value_counts().index, y=train_df['implant'].value_counts().values), row=2, col=2)

fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Count of laterality, density, view and implant', title_x=0.5)
fig.show()

In [36]:
fig = make_subplots(rows=2, cols=2, subplot_titles=('Laterality', 'Density', 'View', 'Implant'))

fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 1]['laterality'].value_counts().index, y=train_df[train_df['cancer'] == 1]['laterality'].value_counts().values, name='Cancer'), row=1, col=1)
fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 0]['laterality'].value_counts().index, y=train_df[train_df['cancer'] == 0]['laterality'].value_counts().values, name='No Cancer'), row=1, col=1)

fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 1]['density'].value_counts().index, y=train_df[train_df['cancer'] == 1]['density'].value_counts().values, name='Cancer'), row=1, col=2)
fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 0]['density'].value_counts().index, y=train_df[train_df['cancer'] == 0]['density'].value_counts().values, name='No Cancer'), row=1, col=2)

fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 0]['view'].value_counts().index, y=train_df[train_df['cancer'] == 0]['view'].value_counts().values, name='No Cancer'), row=2, col=1)
fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 1]['view'].value_counts().index, y=train_df[train_df['cancer'] == 1]['view'].value_counts().values, name='Cancer'), row=2, col=1)

fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 0]['implant'].value_counts().index, y=train_df[train_df['cancer'] == 0]['implant'].value_counts().values, name='No Cancer'), row=2, col=2)
fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 1]['implant'].value_counts().index, y=train_df[train_df['cancer'] == 1]['implant'].value_counts().values, name='Cancer'), row=2, col=2)

fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Count of laterality, density, view and implant with respect to the cancer', title_x=0.5)
fig.show()

# 4. Missing Values 

In [37]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percentage = (data.isnull().sum()/data.isnull().count()).sort_values(ascending = False)
    return pd.concat([total,percentage] , axis = 1 , keys = ['Total' , 'Percent'])

missing_data(train_df)

,Total,Percent
BIRADS,28420,0.519504
density,25236,0.461302
age,37,0.000676
Unnamed: 0,0,0.000000
invasive,0,0.000000
input_image_path,0,0.000000
difficult_negative_case,0,0.000000
machine_id,0,0.000000
implant,0,0.000000
biopsy,0,0.000000


In [38]:
train_df = train_df.drop(['BIRADS'], axis=1)
train_df['age'] = train_df['age'].fillna(train_df['age'].mean())
train_df['density'] = train_df['density'].fillna('E')

In [39]:
missing_data(train_df)

,Total,Percent
Unnamed: 0,0,0.0
site_id,0,0.0
patient_id,0,0.0
image_id,0,0.0
laterality,0,0.0
view,0,0.0
age,0,0.0
cancer,0,0.0
biopsy,0,0.0
invasive,0,0.0


In [40]:
train_df['laterality'] = train_df['laterality'].astype('category')
train_df['view'] = train_df['view'].astype('category')
train_df['age'] = train_df['age'].astype('int64')
train_df['output_image_path'] = train_df['output_image_path'].astype('string')

# 5. Fix Data Types

In [41]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Unnamed: 0               54706 non-null  int64   
 1   site_id                  54706 non-null  int64   
 2   patient_id               54706 non-null  int64   
 3   image_id                 54706 non-null  int64   
 4   laterality               54706 non-null  category
 5   view                     54706 non-null  category
 6   age                      54706 non-null  int64   
 7   cancer                   54706 non-null  int64   
 8   biopsy                   54706 non-null  int64   
 9   invasive                 54706 non-null  int64   
 10  implant                  54706 non-null  int64   
 11  density                  54706 non-null  object  
 12  machine_id               54706 non-null  int64   
 13  difficult_negative_case  54706 non-null  bool    
 14  input_

# 6. Numerical Categorization

In [42]:
train_df["age_bin"] = pd.cut(train_df['age'].values.reshape(-1), bins=5, labels=False)

# 7. Generate Dummy Variables 

In [43]:
cat_cols = ['laterality', 'view', 'density', 'difficult_negative_case']
train_df = pd.get_dummies(train_df, columns=cat_cols)

In [44]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Unnamed: 0                     54706 non-null  int64 
 1   site_id                        54706 non-null  int64 
 2   patient_id                     54706 non-null  int64 
 3   image_id                       54706 non-null  int64 
 4   age                            54706 non-null  int64 
 5   cancer                         54706 non-null  int64 
 6   biopsy                         54706 non-null  int64 
 7   invasive                       54706 non-null  int64 
 8   implant                        54706 non-null  int64 
 9   machine_id                     54706 non-null  int64 
 10  input_image_path               54706 non-null  object
 11  output_image_path              54706 non-null  string
 12  age_bin                        54706 non-null  int64 
 13  l